In [ ]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.5.0`

import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
  .appName("ScalaSpark")
  .master("local[*]")
  .getOrCreate()

import spark.implicits._


In [ ]:
import org.apache.spark.sql.expressions.Aggregator
import org.apache.spark.sql.Encoders
import org.apache.spark.sql.functions.udaf

case class IntSumBuffer(var sum: Long = 0L)

object SumAll extends Aggregator[Int, IntSumBuffer, Long] {
  def zero = IntSumBuffer(0L)
  def reduce(buffer: IntSumBuffer, value: Int): IntSumBuffer = {
    buffer.sum += value
    buffer
  }
  def merge(b1: IntSumBuffer, b2: IntSumBuffer): IntSumBuffer = {
    b1.sum += b2.sum
    b1
  }
  def finish(reduction: IntSumBuffer): Long = reduction.sum
  def bufferEncoder = Encoders.product[IntSumBuffer]
  def outputEncoder = Encoders.scalaLong
}

val sumAll = udaf(SumAll)


In [ ]:
val df = Seq(1, 2, 3, 4, 5).toDF("value")
df.select(sumAll($"value").as("total_sum")).show()